# Time Series Analysis and Forecasting

### Task 1:

Выберите 2 любые компании, которые имеют листинг на фондовом рынке США. Выгрузите дневные данные начиная с 2017 по настоящее время по этим 2-м компаниям и сформируйте датасет для временного ряда (итого у вас 2 датасета). 
<br><b>Цель задания</b> - проведите исследование и статистический анализ временного ряда по сформированным датасетам, посчитайте все основные статистики для временного ряда, превратите ваши временные ряды в стационарные и сформируйте отчет с выводами и визуализациями в jupyter notebook’e.

Без отчета с подкрепленными выводами задание считается проваленным (то есть ноутбук с кодом и картинками мне не присылать).

Итоговая оценка за правильно выполненное задание: <b>3 балла</b>.


<div style='background-color: lightblue; padding: 10px; border-radius: 10px;'>
    <h4 style='color: darkblue;'> yfinance</h4>
</div>

In [ ]:
import yfinance as yf
import pandas as pd

Выберем две компании и скачаем данные с 2017, используя библиотеку yfinance.

In [ ]:
# Компании
companies = {
    'EPAM': 'EPAM Systems, Inc.',
    'AAPL': 'Apple Inc.'
}

In [ ]:
# Даты
start_date = '2017-01-01'
end_date = '2025-04-29'

In [ ]:
# Скачивание данных
data = {}
for ticker in companies:
    df = yf.download(ticker, start=start_date, end=end_date)
    df.to_csv(f'{ticker}_daily_data.csv')  # Сохраняем в файл
    data[ticker] = df

In [ ]:
df.head()

Оставим только цену закрытия торгов, это самый репрезентативный показатель и упростит анализ)

In [ ]:
# Оставляем только закрытие
close_data = {ticker: df[['Close']] for ticker, df in data.items()}

In [ ]:
df = pd.concat(close_data, axis=1)
df.columns = ['EPAM', 'AAPL']

<div style='background-color: lightblue; padding: 10px; border-radius: 10px;'>
    <h4 style='color: darkblue;'> Dataset</h4>
</div>

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.to_csv('closing_price.csv')

У нас теперь есть датафрейм с ценами закрытия акций EPAM и AAPL с 2017 года по сегодняшний день.

In [ ]:
plt.figure(figsize=(14, 6))
# EPAM
plt.plot(df.index, df['EPAM'], label='EPAM', alpha=0.7)
# AAPL
plt.plot(df.index, df['AAPL'], label='AAPL', alpha=0.7)
plt.title('Закрытые цены акций EPAM и AAPL (2017-2025)', fontsize=16)
plt.xlabel('Дата')
plt.ylabel('Цена закрытия (USD)')
plt.legend()
plt.grid(True)
plt.show()

<div style='background-color: lightblue; padding: 10px; border-radius: 10px;'>
    <h4 style='color: darkblue;'> Statistics</h4>
</div>

In [ ]:
df.describe()

In [ ]:
stats = df.describe().T
print(f'coef_var: \n{stats['std'] / stats['mean']}')

In [ ]:
df_ma = df.copy()
df_ma['EPAM_MA252'] = df['EPAM'].rolling(window=252).mean()
df_ma['AAPL_MA252'] = df['AAPL'].rolling(window=252).mean()

Используем скользящее среднее для выделения тренда. 252 биржевых дня - календарный биржевой год.

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(df_ma.index, df_ma['EPAM'], label='EPAM', alpha=0.6)
plt.plot(df_ma.index, df_ma['AAPL'], label='AAPL', alpha=0.6)
plt.plot(df_ma.index, df_ma['EPAM_MA252'], label='EPAM (MA252)', linestyle='--')
plt.plot(df_ma.index, df_ma['AAPL_MA252'], label='AAPL (MA252)', linestyle='--')

plt.title('Закрытые цены и тренд (MA252) EPAM и AAPL (2017–2025)')
plt.xlabel('Дата')
plt.ylabel('Цена закрытия (USD)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

<br>Средняя цена:
 - EPAM: 251.41 USD
 - AAPL: 115.65 USD

<br>Стандартное отклонение (разброс цен):
 - EPAM: 138.98
 - AAPL: 65.08

<br>Минимум - максимум:
 - EPAM: от 63.66 до 717.49 USD
 - AAPL: от 26.83 до 258.74 USD

<br>Коэффициент вариации — показатель относительной волатильности:
 - EPAM: 0.55
 - AAPL: 0.56

EPAM демонстрировал очень сильный рост до 2022 года, но затем вошёл в затяжной нисходящий тренд, отчасти сохранив высокую волатильность. AAPL показывает устойчивое и стабильное восходящее движение с умеренной волатильностью.

<div style='background-color: lightblue; padding: 10px; border-radius: 10px;'>
    <h4 style='color: darkblue;'> Augmented Dickey-Fuller test</h4>
</div>

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
def adf_test(series, title=''):
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Statistic', 'p-value', 'Lags Used', 'Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    return out

In [ ]:
adf_results = {
    'EPAM': adf_test(df['EPAM']),
    'AAPL': adf_test(df['AAPL'])
}

In [ ]:
pd.DataFrame(adf_results)

p-value > 0.05 для обеих компаний  -  ряд нестационарный. Это ожидаемо, потому что на графиках был виден явный тренд.

<div style='background-color: lightblue; padding: 10px; border-radius: 10px;'>
    <h4 style='color: darkblue;'> Stationary</h4>
</div>

In [ ]:
# Приведение к стационарному виду — первое дифференцирование
df_diff = df.diff().dropna()

In [ ]:
# Повторный тест Дики-Фуллера
adf_results_diff = {
    'EPAM_diff': adf_test(df_diff['EPAM']),
    'AAPL_diff': adf_test(df_diff['AAPL'])
}
pd.DataFrame(adf_results_diff)

p-value < 0.05 - ряд стационарный для обоих компаний

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(df_diff.index, df_diff['EPAM'], label='EPAM (diff)', alpha=0.7)
plt.plot(df_diff.index, df_diff['AAPL'], label='AAPL (diff)', alpha=0.7)
plt.title('Стационарные ряды')
plt.xlabel('Дата')
plt.ylabel('Разность закрытия')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

<div style='background-color: lightblue; padding: 10px; border-radius: 10px;'>
    <h4 style='color: darkblue;'> Auto correlations</h4>
</div>

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
# EPAM ACF и PACF
plot_acf(df_diff['EPAM'], ax=axes[0, 0], lags=40, title='EPAM - ACF')
plot_pacf(df_diff['EPAM'], ax=axes[0, 1], lags=40, title='EPAM - PACF', method='ywm')
# AAPL ACF и PACF
plot_acf(df_diff['AAPL'], ax=axes[1, 0], lags=40, title='AAPL - ACF')
plot_pacf(df_diff['AAPL'], ax=axes[1, 1], lags=40, title='AAPL - PACF', method='ywm')
plt.tight_layout()
plt.show()

1. ACF (Autocorrelation Function) - полная автокорреляция. Показывает насколько текущее значение зависит от прошлых значений на разных лагах.
    - EPAM: ACF затухает постепенно, 1 и 2 лаг положительный, 3- отрицательный.
    - AAPL: ACF  обрывается быстро.
3. PACF (Partial Autocorrelation Function — частичная автокорреляция. Показывает связь между текущим значением и лагом, не обращая внимания на промежуточные.
    - EPAM: PACF показывает значимость на 1–2 лагах, затем обрывается.
    - AAPL: PACF обрывается быстро.

<div style='background-color: lightblue; padding: 10px; border-radius: 10px;'>
    <h4 style='color: darkblue;'> Seasonality</h4>
</div>

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# Период 252 — примерно количество торговых дней в году
decomposition_epam = seasonal_decompose(df['EPAM'], model='additive', period=252)
decomposition_aapl = seasonal_decompose(df['AAPL'], model='additive', period=252)

In [ ]:
fig_epam = decomposition_epam.plot()
fig_epam.set_size_inches(14, 8)
fig_epam.suptitle('Сезонная декомпозиция EPAM (период=252)', fontsize=16)
plt.tight_layout()
plt.show()

Trend:
 - С 2017 до конца 2021 наблюдается устойчивый рост.
 - После пика в начале 2022 года — тренд сменился на спад, что может быть связано с рыночными/внешними событиями (например, геополитика, санкции и т.д.).

Seasonal:
 - Есть повторяющаяся структура.
 - Сезонность стабильна по амплитуде (80 USD), не растёт и не падает вместе с трендом.

Residual:
 - Остатки довольно волатильны (особенно в 2022 году).
 - Это говорит о том, что в ряду есть внезапные отклонения, не объяснимые ни трендом, ни сезонностью.

In [ ]:
fig_aapl = decomposition_aapl.plot()
fig_aapl.set_size_inches(14, 8)
fig_aapl.suptitle('Сезонная декомпозиция AAPL (период=252)', fontsize=16)
plt.tight_layout()
plt.show()

Trend:
 - Чётко выражен устойчивый рост с 2017 до 2022.
 - С 2022 рост замедляется, но продолжается до 2025.
 - Длинный растущий тренд.

Seasonal:
 - Есть годовая периодичность.
 - Сезонность стабильна по амплитуде (20 USD), не растёт и не падает вместе с трендом.

 Residual (остатки):
 - Распределены умеренно, но видно волатильность — особенно в 2022–2023 годах.
 - Это может отражать рыночные события, новости, отчёты, санкции и пр.

Обе акции показывают общий тренд роста, но видно, что в разные периоды были спады и волатильность. AAPL показывает устойчивый рост без резких обвалов, даже в моменты рыночной нестабильности. Период 2022–2024 у AAPL наблюдается коррекция, но она более плавная и контролируемая, чем у EPAM. С 2020 по начало 2022 года акции EPAM выросли в несколько раз. Это может быть связано с пандемийным ИТ-бумом и высокой технологической востребованностью услуг компании. Однако в 2022 году последовал резкий обвал вероятно вероятно связанный с геополитическими рисками, переоценкой ИТ-компаний, общим спадом в секторе после пандемии. К 2025 году AAPL и EPAM находятся примерно на одном уровне по цене закрытия, хотя раньше EPAM значительно опережала. Возможно компания AAPL в прошлом была недооценена, и теперь цена акции корректируется, так же возможно, что компания EPAM была переоценена и также цена за акцию подвергается коррекции. 

# Task 2:

Выберите <b>1</b> из 2-ух временных рядов <b>из задания 1</b> и сделайте <b>прогноз на ближайший месяц-квартал-полугодие  и год</b>. Модель выбирать исходя из ваших предположений и умозаключений. Необходимо обосновать почему была выбрана та или иная модель и на каких основаниях, в чем она лучше и почему она подходит. Оценить качество прогноза согласно выбранной метрике, проверить качество на отложенной выборке путем применения TimeSeriesSplit. Сформировать отчет с выводами и визуализациями в jupyter notebook’e.

Без отчета с подкрепленными выводами задание считается проваленным (то есть ноутбук с кодом и картинками мне не присылать).

Итоговая оценка за правильно выполненное задание: <b>3 балла.</b>


<div style='background-color: lightblue; padding: 10px; border-radius: 10px;'>
    <h4 style='color: darkblue;'> LinearRegression</h4>
</div>

Для начала выберем линейную регрессию для прогнозирования временного ряда стоимости акциий компании Apple. Она хорошо подходит при наличии тренда, а так же быстро работает, не требует подбора гипер-параметров , что удобно при использовании кросс-валидации. Можно так же использовать как часть ансамбля для определения тренда, а в связке с XGBoost — для корректировки остатков.

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
series = df['AAPL']

In [ ]:
lags = [1, 2, 3]

In [ ]:
features = pd.DataFrame({'y': series})

In [ ]:
# Лаги
for lag in lags:
    features[f'lag_{lag}'] = series.shift(lag)

In [ ]:
# Удаление строк с NaN после генерации признаков
features = features.dropna()

In [ ]:
# Разделение X и y
X = features.drop(columns=['y'])
y = features['y']

In [ ]:
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits) 

In [ ]:
rmse_scores = []
mae_scores = []
mape_scores = []
r2_scores = []
y_true_all = pd.Series(dtype=float)
y_pred_all = pd.Series(dtype=float)

In [ ]:
y_true_all = pd.Series(dtype=float)
y_pred_all = pd.Series(dtype=float)

for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Сохраняем предсказания с правильным индексом
    y_pred_all = pd.concat([y_pred_all, pd.Series(y_pred, index=y_test.index)])
    y_true_all = pd.concat([y_true_all, y_test])

    # Метрики (оставим как есть)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)

    rmse_scores.append(rmse)
    mae_scores.append(mae)
    mape_scores.append(mape)
    r2_scores.append(r2)

    print(f"Fold {fold+1}: RMSE={rmse:.2f}, MAE={mae:.2f}, MAPE={mape:.2f}%, R2={r2:.2f}")

In [ ]:
pd.DataFrame({
    'Fold': [f'Fold {i+1}' for i in range(n_splits)],
    'RMSE': rmse_scores,
    'MAE': mae_scores,
    'MAPE (%)': mape_scores,
    'R2': r2_scores
})

 - Все метрики демонстрируют низкие ошибки.
 - Очень высокие значения R² (близкие к 1) на каждом фолде — признак того, что признаки (лаги) хорошо объясняют поведение ряда.
 - MAPE < 2 % говорит о высокой точности относительно текущих цен — особенно важно в задачах трейдинга или управления рисками.

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(y_true_all.index, y_true_all, label='Истинные значения', color='red')
plt.plot(y_pred_all.index, y_pred_all, label='Прогноз (Linear Regression)', linestyle='--', color='blue')
plt.title('Прогноз vs Истинные значения (Linear Regression, CV)')
plt.xlabel('Дата')
plt.ylabel('Цена (USD)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

<div style='background-color: lightblue; padding: 10px; border-radius: 10px;'>
    <h4 style='color: darkblue;'> Forecast</h4>
</div>

In [ ]:
model_final = LinearRegression()
model_final.fit(X, y)

In [ ]:
# Обучение модели Linear Regression
model = LinearRegression()
model.fit(X, y)

In [ ]:
# Прогнозы на разные горизонты
forecast_horizon = {
    '1 месяц': 21,
    '1 квартал': 63,
    '0.5 года': 126,
    '1 год': 252
}

In [ ]:
# Собираем прогнозы
last_known = features.iloc[-1]
future_forecasts = []

for label, horizon in forecast_horizon.items():
    preds = []
    last_vals = list(last_known[[f'lag_{lag}' for lag in lags]])

    for _ in range(horizon):
        input_vec = np.array([last_vals[-lag] for lag in lags]).reshape(1, -1)
        next_val = model.predict(input_vec)[0]
        preds.append(next_val)
        last_vals.append(next_val)

    forecast_index = pd.date_range(start=series.index[-1], periods=horizon+1, freq='B')[1:]
    future_forecasts.append((label, forecast_index, preds))

In [ ]:
for label, forecast_index, preds in future_forecasts:
    plt.figure(figsize=(12, 4))
    plt.plot(series[-126:], label='Факт (последние 6 мес.)', color='black')
    plt.plot(forecast_index, preds, label=f'Прогноз: {label}', linestyle='--', color='blue')
    plt.title(f'Прогноз AAPL на {label} (Linear Regression)')
    plt.xlabel('Дата')
    plt.ylabel('Цена (USD)')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

Во всех горизонтах модель выдает плавный, монотонно растущий прогноз. Линейная регрессия не захватывает сложные паттерны в поведении цены акций. После резкого падения в марте–апреле 2025 модель не прогнозирует сильного восстановления, хотя данные намекают на возможный рост.

<div style='background-color: lightblue; padding: 10px; border-radius: 10px;'>
    <h4 style='color: darkblue;'> Prophet</h4>
</div>

Когда стоит использовать Prophet:
 - Нужен быстрый базовый прогноз, без ручной настройки.
 - Данные содержат сезонные колебания или тренд.

In [ ]:
from prophet import Prophet

In [ ]:
series = df['AAPL']
# Prophet требует DataFrame с колонками 'ds' и 'y'
df_prophet = pd.DataFrame({'ds': series.index, 'y': series.values})

In [ ]:
# TimeSeriesSplit
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)

In [ ]:
rmse_scores, mae_scores, mape_scores, r2_scores = [], [], [], []
predictions = pd.Series(index=series.index, dtype=float)

In [ ]:
for fold, (train_idx, test_idx) in enumerate(tscv.split(df_prophet)):
    train = df_prophet.iloc[train_idx].copy()
    test = df_prophet.iloc[test_idx].copy()

    model = Prophet(daily_seasonality=False, weekly_seasonality=False, 
                    yearly_seasonality=True, seasonality_mode='multiplicative')
    model.fit(train)

    future = test[['ds']]
    forecast = model.predict(future)
    y_true = test['y'].values
    y_pred = forecast['yhat'].values
    predictions.iloc[test_idx] = y_pred

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)

    rmse_scores.append(rmse)
    mae_scores.append(mae)
    mape_scores.append(mape)
    r2_scores.append(r2)

    print(f"Fold {fold+1}: RMSE={rmse:.2f}, MAE={mae:.2f}, MAPE={mape:.2f}%, R2={r2:.2f}")

In [ ]:
pd.DataFrame({
    'Fold': [f'Fold {i+1}' for i in range(n_splits)],
    'RMSE': rmse_scores,
    'MAE': mae_scores,
    'MAPE (%)': mape_scores,
    'R2': r2_scores
})

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(series, label='Факт', color='red')
plt.plot(predictions, label='Prophet прогноз (CV)', linestyle='--', color='green')
plt.title('Прогноз AAPL с Prophet и TimeSeriesSplit')
plt.xlabel('Дата')
plt.ylabel('Цена (USD)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

<div style='background-color: lightblue; padding: 10px; border-radius: 10px;'>
    <h4 style='color: darkblue;'> Forecast</h4>
</div>

Prophet — это мощный инструмент для прогнозирования будущих значений в данных временных рядов, особенно когда ваши данные имеют повторяющиеся закономерности, такие как ежемесячные или годовые циклы.

In [ ]:
# Обучение модели Prophet
model = Prophet(daily_seasonality=False, weekly_seasonality=False, 
                    yearly_seasonality=True, seasonality_mode='multiplicative')
model.fit(df_prophet)

In [ ]:
for label, horizon in forecast_horizon.items():
    future = model.make_future_dataframe(periods=horizon, freq='B')
    forecast = model.predict(future)
    
    plt.figure(figsize=(12, 4))
    plt.plot(df_prophet['ds'][-126:], df_prophet['y'][-126:], label='Факт (последние 6 мес.)', color='red')
    plt.plot(forecast['ds'][-horizon:], forecast['yhat'][-horizon:], label=f'Прогноз: {label}', linestyle='--', color='green')
    plt.title(f'Прогноз AAPL на {label} (Prophet)')
    plt.xlabel('Дата')
    plt.ylabel('Цена (USD)')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
for label, horizon in forecast_horizon.items():
    future = model.make_future_dataframe(periods=horizon, freq='B')
    forecast = model.predict(future)

    plt.plot(df_prophet['ds'], df_prophet['y'], label='Факт')
    plt.plot(forecast['ds'], forecast['yhat'], label='Прогноз')
    plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], alpha=0.3)
    plt.legend()
    plt.grid(True)
    plt.title(f'Сравнение прогноза Prophet с фактами ({label})')
    plt.show()


Впрогнозе на первый месяц видим умеренный рост, в 1 квартале продолжается восходящий тренд, 6 месяцев - устойчивый рост, 1 год - продолжение тренда.
Несмотря на явное снижение в марте–апреле 2025, Prophet не закладывает его в прогноз, демонстрирует слабую волатильность и периодические колебания. Модель предсказывает устойчивый рост цены AAPL. 

Прогнозная линия (оранжевая) следит за историческим ростом цены AAPL довольно точно. Фактические значения в большинстве случаев укладываются в доверительный интервал, что подтверждает надежность прогноза.

In [ ]:
model.plot_components(forecast)
plt.suptitle(f"Компоненты прогноза Prophet ({label})", fontsize=14)
plt.tight_layout()
plt.show()

Тренд (trend component):
 - 	С 2017 по 2025 цена AAPL демонстрирует устойчивый восходящий тренд.
 - 	В 2022–2023 наблюдается временное замедление роста (возможно, связано с внешними событиями).
 - 	В 2025–2026 модель прогнозирует восстановление и дальнейший рост.
 - 	Интервал неопределенности расширяется в будущем, что естественно при долгосрочном прогнозировании.

Годовая сезонность (yearly component):
 - Prophet выявил сезонную структуру доходности в течение года:
 - Апрель–май — исторически слабые месяцы: наблюдается спад.
 - Август–сентябрь — подъём.
 - Декабрь и начало января — снова заметный рост (вероятно, рождественский эффект или период годовой отчетности).

Инвестиционные рекомендации - покупка:
 - модель Prophet прогнозирует устойчивый рост в течение следующего полугодия и года.
 - Подходит для долгосрочных инвесторов, ориентированных на рост стоимости.
 - Продажа в текущей точке могла бы упустить потенциальную прибыль на росте.